In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from time import sleep

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
#Values Corresponding to each league, used in Fishy's URLs
league_dict = {'Premier League': 1, 'League 1': 34, 'Bundesliga': 32, 'Serie A': 33, 'La Liga': 31}

In [4]:
#Get a Final League Table from any top 5 league since 2005 
def get_table(year, league):
    #Grab url
    url = 'https://thefishy.co.uk/leaguetable.php?table={}&season={}'.format(league_dict[league], year-2002)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    #If the league is bundesliga - only 18 teams
    num_teams = 20
    if league == 'Bundesliga':
        num_teams = 18
    
    #Grab HTML corresponding to each team name and their point value
    team_scripts = soup.find_all('td', {'class': 'cats'})[:num_teams]
    
    #General area in which the point tallys reside within the HTML code
    point_scripts = soup.find_all('td')
    
    #create team list from the scripts
    teams = []
    for i in team_scripts:
        teams.append(i.string) 
    
    #Create points list, max_scripts refers to how many lines of code to parse through
    points = []
    max_scripts = 425
    if league == 'Bundesliga':
        max_scripts = 383
    #Every 21st line of HTML code holds the point tally, parse through and add them to the points list
    for i in range(23, max_scripts, 21):
        points.append(point_scripts[i].string)
    
    #Create simple dataframe
    df = pd.DataFrame(list(zip(teams, points)),
               columns =['team', 'points'])
    df['league'] = league
    df['year'] = year
    return df
    

In [7]:
#Example
get_table(2008, 'Premier League')

,team,points,league,year
0,Man Utd,90,Premier League,2008
1,Liverpool,86,Premier League,2008
2,Chelsea,83,Premier League,2008
3,Arsenal,72,Premier League,2008
4,Everton,63,Premier League,2008
5,Aston Villa,62,Premier League,2008
6,Fulham,53,Premier League,2008
7,Tottenham,51,Premier League,2008
8,West Ham,51,Premier League,2008
9,Man City,50,Premier League,2008
